In [65]:
import cv2
import numpy as np
import typing
from matplotlib import pyplot as plt
import imutils

import polarTransform as polTrans

import os
import math
import time

In [66]:
# constants
tumbnail_size = 300

working_p = "./dataset/"
in_p = working_p + "canon/"
out_p = working_p + "evaluation/"

In [67]:
# import from in_p folder all file names as paths
files = [f for f in os.listdir(in_p)]

In [68]:
def save_img_in_its_folder(img, out, file):
    file_name = file
    if file.count(".") > 0:
        file_name = ".".join([x for x in file.split(".")[:-1]])
    
    path = out + file_name + "/"
    if not os.path.exists(path):
        os.makedirs(path)
    
    cv2.imwrite(path + file_name + ".png", img)


In [69]:
def scale_to(img, size: int, interpolation=cv2.INTER_AREA):
    """
    Resize square image to given size (in pixels)
    """
    return cv2.resize(img, (size, size), interpolation=interpolation)

In [70]:
# 1. Load image
def load_img(file) -> np.ndarray:
    """
    load original image and return
    @param <IN> file: file name (not the path)

    @return <OUT> image
    """
    
    path = in_p + file
    img = cv2.imread(path)

    # crop img to square and keep the middle in the middle
    h, w = img.shape[:2]
    t = min(h, w)
    img = img[(h-t)//2:(h+t)//2, (w-t)//2:(w+t)//2]

    return img

def process(img, clahe):
    """
    Process image
    """
    # use clahe
    img_p = clahe.apply(img)
    return img_p

In [71]:
# 2. find all other circles around the first found circle by adjusting the min and max radius
def get_circles_around_main(img, c_list: typing.Set[typing.Tuple[int]],
        mc, param1: int, param2: int, min_dist: int, min_r: int, max_r: int,
        m_offset: int, e_offset: int, step: int) -> None:
    """
    Get circles around main circle
    @param <IN> img: image
    @param <OUT> c_list: list of circles
    @param <IN> mc: main circle
    @param <IN> param1: first parameter of cv2.HoughCircles
    @param <IN> param2: second parameter of cv2.HoughCircles
    @param <IN> min_dist: minimum distance between circles
    @param <IN> min_r: min radius
    @param <IN> max_r: max radius
    @param <IN> m_offset: offset from main circle
    @param <IN> e_offset: offset from min_r and max_r
    @param <IN> step: step (should be a pozitive integer)
    """

    for new_min_r in range(mc[2] + m_offset, max_r - e_offset, step):
        circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 2,
                                    param1=param1, param2=param2,
                                    minDist=min_dist,
                                    minRadius=new_min_r, maxRadius=max_r)
        if circles is not None:
            circle = circles[0][0]
            circle = np.int32(np.around(circle))
            c_list.add(tuple(circle))

    for new_max_r in range(mc[2] - m_offset, min_r + e_offset, -step):
        circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 2,
                                    param1=param1, param2=param2,
                                    minDist=min_dist,
                                    minRadius=min_r, maxRadius=new_max_r)
        if circles is not None:
            circle = circles[0][0]
            circle = np.int32(np.around(circle))
            c_list.add(tuple(circle))


In [72]:
# 3. Reduce circles that are very simmilar and keep a score for how many were combined
def reduce_circles(c_list: typing.Set[typing.Tuple[int]],
        cd: int, rd: int) -> typing.List[typing.Tuple[typing.Tuple[int], int]]:
    """
    Reduce circles that are very simmilar and keep a score for how many were combined
    @param <IN> c_list: set of circles
    @param <IN> cd: center distance
    @param <IN> rd: radius difference

    @return <OUT> c_list: list of tuples (circles, how many were combined to create it)
    """

    ret = []

    copy = list(c_list)

    index: int = 0
    while index < len(copy):
        (x1, y1, r1) = copy[index]
        final_props = [(x1, y1, r1)]

        poped: bool = True
        while poped == True:
            poped = False

            for i in range(index + 1, len(copy)):
                (x2, y2, r2) = copy[i]

                # print("index:", index, "| i:", i, "| dist:", math.dist((x1, y1), (x2, y2)), "| r_diff:", abs(r1 - r2))
                if (math.dist((x1, y1), (x2, y2)) <= cd) and (abs(r1 - r2) <= rd):
                    poped = True
                    final_props.append((x2, y2, r2))
                    # calculate a middle from where to search furether centers
                    x1 = (x1 + x2) / 2
                    y1 = (y1 + y2) / 2
                    r1 = (r1 + r2) / 2

                    copy.pop(i)
                    break
            # for END
        # while END

        # use the mean to calculate a new circle
        x_r: int = 0
        y_r: int = 0
        r_r: int = 0
        for (x, y, r) in final_props:
            x_r += x
            y_r += y
            r_r += r

        x_r /= len(final_props)
        y_r /= len(final_props)
        r_r /= len(final_props)

        ret.append(((int(x_r), int(y_r), int(r_r)), len(final_props)))

        index += 1

    return ret

In [73]:
# 4. creates 2 concentric circles
def get_concentric_circles(cl_n_f: typing.List[typing.Tuple[typing.Tuple[int], int]]) -> typing.List[typing.Tuple[int]]:
    """
    Selects the pair of circles with centers close to each other.
    Then we create the new concentric outer circle based on the inner one
    @param <IN> cl_n_f: list of tuples (circles, how many were combined to create it)

    @return <OUT> c_list: list of circles [small_circle, big_circle]
    Note: For now we accept only 2 circles per image
    """

    if len(cl_n_f) != 2:
        return [x for x, _ in cl_n_f]

    c_list: typing.List[typing.Tuple[int]] = []
    (x1, y1, r1) = cl_n_f[0][0]
    (x2, y2, r2) = cl_n_f[1][0]

    # we base the bigger circle on the smaller one because experiments show that the smaller one is better calculated ussually
    d: int = int(math.dist((x1, y1), (x2, y2)))

    if (r1 < r2):
        # inner circle
        c_list.append((x1, y1, r1))
        # calculate the new outer circle radius
        c_list.append((x1, y1, r2 + d))
    else:
        # inner circle
        c_list.append((x2, y2, r2))
        # calculate the new outer circle radius
        c_list.append((x2, y2, r1 + d))

    return c_list

In [74]:
def get_tire_circles(img, min_r: int, max_r: int, min_dist: int, param1: int, param2: int):
    """
    Find the outter circle and the inner circle in an image with a tire
    1. find the first circle in the image
    2. find all other circles around the first found circle by adjusting the min and max radius
    3. reduce circles that are very simmilar and keep a score for how many were combined
    4. creates 2 concentric circles

    Note: min_dist should be img size so we find just a circle in the image
    """

    # 1. find the first circle in the image
    circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 2,
                                param1=param1, param2=param2,
                                minDist=min_dist,
                                minRadius=min_r, maxRadius=max_r)
    
    if circles is None:
        circles = np.array([[[img.shape[0] / 2, img.shape[1] / 2, min(img.shape[0], img.shape[1]) / 3]]])
        print("No circles found, substituting with center of image")

    start_circle = circles[0][0]
    start_circle = np.int32(np.around(start_circle))

    circle_l: typing.Set[typing.Tuple[int]] = set()
    circle_l.add(tuple(start_circle))
    
    # print("circle_l: ", circle_l, end="\n\n")

    # 2. find all other circles around the first found circle by adjusting the min and max radius
    get_circles_around_main(img, circle_l, start_circle, param1, param2, min_dist, min_r, max_r, 10, 5, 10)

    # 3. remove circles that are very simmilar, but keep a score for the most propable circle
    # circle and frequency
    cl_n_f = reduce_circles(circle_l, cd = 20, rd = 15)

    # 4. creates 2 concentric circles
    circle_l = get_concentric_circles(cl_n_f)

    return circle_l
    

In [75]:
# 5
def scale_circles(
        cs: typing.List[typing.Tuple[int, int, int]],
        shape: typing.Tuple[int, int],
        target_shape: typing.Tuple[int, int]
        ) -> typing.List[typing.Tuple[int, int, int]]:
    """
    Scales the circle properties to the target shape

    @param <IN> cs: list of circles
    @param <IN> shape: original image shape (height, width)
    @param <IN> target_shape: target image shape (height, width)

    @return <OUT> cs: list of circles scaled

    Note: shape and target_shape height and width ratios must be the same
    """

    # calculate the ratios shape / target_shape and make the check
    r1 = target_shape[0] / shape[0]
    r2 = target_shape[1] / shape[1]

    if (round(r1, 2) != round(r2, 2)):
        raise Exception("The ratios of the original and target image must be the same")
    
    # calculate the new circle properties
    ret: typing.List[typing.Tuple[int, int, int]] = []
    for c in cs:
        x = int(c[0] * r1)
        y = int(c[1] * r1)
        r = int(c[2] * r1)
        ret.append((x, y, r))

    return ret

# 6
def crop_tire(
        img,
        sc: typing.Tuple[int, int, int],
        bc: typing.Tuple[int, int, int]
        ) -> typing.Tuple[typing.Any, typing.List[typing.Tuple[int, int, int]]]:
    """
    Crops from the image only the tire according to the biggest circle.
    Then calculates the new coordinates of the circles in the resulting image.
    
    @param <IN> img: image to crop
    @param <IN> sc: small circle
    @param <IN> bc: big circle

    @return <OUT> (img_crop, circles): tuple of cropped image and list of circles sorted ascending by range 
    """


    new_circles: typing.List[typing.Tuple[int]] = [(), ()]

    (_, _, r_s) = sc
    (x_b, y_b, r_b) = bc
    
    img_h: int = img.shape[0]
    img_w: int = img.shape[1]

    # find the upper corner of the new image
    (x_up, y_up) = (max(int(x_b - r_b), 0), max(int(y_b - r_b), 0))
    # find the lower corner of the new image
    (x_down, y_down) = (min(x_b + r_b, img_w), min(y_b + r_b, img_h))

    # crop the image after the upper and lower corners
    img_crop = img[y_up:y_down, x_up:x_down]

    # center of the new image
    (x_c, y_c) = (r_b, r_b)

    new_circles[0] = (x_c, y_c, r_s)
    new_circles[1] = (x_c, y_c, r_b)

    return (img_crop, new_circles)

In [76]:
# 7. convert from polar coordinates to cartesian coordinates
def unwrap(name: str, img: np.ndarray, circles: typing.List[typing.Tuple[int, int, int]], interpolation: int) -> np.ndarray:
    ((x0, y0, r0), (_, _, r1)) = circles

    resulting_image, settings = polTrans.convertToPolarImage(
        img, center=(x0, y0),
        initialRadius=r0, finalRadius=r1,
        hasColor=True, order=interpolation,
        useMultiThreading=True
    )

    # rotate image 90 degrees counter clockwise
    resulting_image = cv2.rotate(resulting_image, cv2.ROTATE_90_COUNTERCLOCKWISE)

    # cv2.imwrite(out_p + name.split(".")[0] + "-" + str(interpolation) + "." + name.split(".")[-1], resulting_image)
    return resulting_image


1 - Load image

2 - Reduce size to `tumbnail_size` and convert to grayscale

3 - Apply Median Blur on the image to remove noise

-  Extra: Apply Canny edge detection like it would be applied in the circle detection step to see what the algorithm sees

4 - `get_tire_circles`

  4.1 - Find the most proeminent circle in the image, with a radius in the specified range

  4.2 - `get_circles_around_main` uses the radius of the most proeminent circle plus an incrementing offset as the new limit of the radius range when searching for bigger and smaller circles in the image
  
  4.3 - `reduce_circles` uses `cd` and `rd` to determine if 2 circles are simmilar enough to be combined into a single one
  
  4.4 - `get_concentric_circles` in images where we could reduce them to only 2 circles, takes the smaller circle as truth and creates a bigger circle with the same center as the smallest one and radius = d(center_small_circle_, center_big_circle_) + r_big_circle (the order in the returned vector is [small_circle, big_circle])

5 - `scale_circles` scales the circle coordinates to the targeted image size 

6 - `crop_tire` crops tire from image and recalculates the centers

In [92]:
# (file_name, image, circles)
# data: typing.List[typing.Tuple[str, np.ndarray, typing.List[typing.Tuple[int, int, int]]]] = []
clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(64,64))
tumbnail_size = 300

found: int = 0
times = []
pixels_reduced = []

for file in files:
    # 1. Load image and return it
    img_BGR = load_img(file)

    time_start = time.time()

    # 2. preprocess and scale-down
    gray = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2GRAY)
    gray = process(gray, clahe)
    gray_small = scale_to(gray, tumbnail_size)

    # process the original image after the gray one because it gives better results
    h, s, v = cv2.split(cv2.cvtColor(img_BGR, cv2.COLOR_BGR2HSV))
    v = process(v, clahe)
    img_BGR = cv2.cvtColor(np.dstack((h, s, v)), cv2.COLOR_HSV2BGR)

    # 3. smooth the image (aka remove noise by blur)
    # gray_small_blur = cv2.GaussianBlur(gray_small, (5, 5), cv2.BORDER_DEFAULT)
    gray_small_blur = cv2.medianBlur(gray_small, 5)

    higher_pass = 140
    
    # extra Canny edge detection
    # gray_small_blur_canny = cv2.Canny(gray_small_blur, higher_pass / 2, higher_pass, L2gradient=True)

    # cv2.imwrite(out_p + file_name + "-0", gray_small_blur_canny)

    # 4. find the circles in the image
    circles = get_tire_circles(gray_small_blur, 70, 140, tumbnail_size, higher_pass, 130)
    # print("circles of", file_name, ":", circles)

    # # DEBUG
    # # print the circles in the image
    # for c in circles:
    #     cv2.circle(gray_small, (c[0], c[1]), c[2], (0, 255, 0), 2)
    
    # cv2.imwrite(out_p + file.split('.')[0] + "-temp." + file.split('.')[-1], gray_small)

    # consider only immages with 2 circles
    if len(circles) != 2:
        continue

    # 5. scale the circle coordinates to the original image size
    circles = scale_circles(circles, gray_small.shape, img_BGR.shape)

    # 6. crop tire from image and recalculate the centers
    (tire_img, circles) = crop_tire(img_BGR, circles[0], circles[1])
    unwrapped = unwrap(file, tire_img, circles, interpolation = 1)
    found += 1

    time_end = time.time()
    times.append(time_end - time_start)

    save_img_in_its_folder(unwrapped, out_p, file)
    pixels_reduced.append(img_BGR.shape[0] * img_BGR.shape[1] - unwrapped.shape[0] * unwrapped.shape[1])
    
    # -------------------------------

    # output_img = tire_img.copy()
    # for (x, y, r) in circles:
        # print(file_name, "->", circle)
        # cv2.circle(output_img, (x, y), r, (0, 255, 0), 2)
        # cv2.rectangle(output_img, (x - 5, y - 5), (x + 5, y + 5), (0, 255, 0), 1)

    # cv2.imwrite(out_p + file_name + "-1", output_img)
    
    # break # only for testing

print("tires unwrapped:", found)
print("times: ", times)
print("pixels_reduced:", pixels_reduced)

# print times in a file
with open(out_p + "times-stage1.txt", "w") as f:
    f.write(str(times))

tires unwrapped: 86
times:  [1.8600032329559326, 1.9210388660430908, 1.753812551498413, 1.5858104228973389, 1.3645086288452148, 1.247645378112793, 1.4841012954711914, 1.7125825881958008, 0.5910718441009521, 1.7043988704681396, 0.6765153408050537, 1.5630862712860107, 3.038506031036377, 1.142134189605713, 1.4061672687530518, 1.608339548110962, 1.2149224281311035, 1.885030746459961, 1.214301347732544, 1.2574338912963867, 1.1533305644989014, 1.3510606288909912, 1.3672046661376953, 1.189260721206665, 1.1561954021453857, 1.7278504371643066, 1.5881829261779785, 1.4816770553588867, 3.1972672939300537, 1.665043592453003, 2.64794921875, 1.4884552955627441, 1.5472981929779053, 1.382373571395874, 1.4894914627075195, 1.2321550846099854, 2.156550884246826, 1.9198901653289795, 1.76137375831604, 1.3000166416168213, 1.5725550651550293, 1.5118908882141113, 1.3664252758026123, 1.9838502407073975, 1.156348466873169, 1.633413314819336, 1.651808738708496, 1.6083660125732422, 1.3628973960876465, 1.3239772319